In [ ]:
!pip install spacy

In [1]:
!python -m spacy download pt_core_news_lg

2023-11-24 20:27:44.305058: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 20:27:44.305123: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 20:27:44.305164: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 20:27:44.343492: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 20:27:47.924449: W tensorflow/c

Dataset disponível em: https://www.gov.br/trabalho-e-emprego/pt-br/assuntos/inspecao-do-trabalho/areas-de-atuacao/combate-ao-trabalho-escravo-e-analogo-ao-de-escravo

In [2]:
import spacy

nlp = spacy.load("pt_core_news_lg")  # Modelo portugues large


In [ ]:
# Frase de entrada
frase = "FAZENDA NOSSA SENHORA DAS GRAÇAS, ZONA RURAL, PEDREGULHO/SP"

# Processar a frase com spaCy
doc = nlp(frase)

# Iterar pelas entidades nomeadas no texto
for entidade in doc.ents:
    if entidade.label_ == "LOC":  # Localidades (cidade, estado, rua...)
        cidade = entidade.text
        print("Localidade:", cidade)


Localidade: PEDREGULHO
Localidade: SP


In [ ]:
frase = "CERÂMICA J.A, RODOVIA PI, SENTIDO DE BARRAS A CABACEIRAS, KM 7, Nº S/N, ZONA RURAL, BARRAS/PI. "

doc = nlp(frase)

for entidade in doc.ents:
      print("palavra:", entidade.text)
      print("entidade:", entidade.label_)


palavra: RODOVIA PI
entidade: LOC
palavra: SENTIDO
entidade: LOC
palavra: CABACEIRAS
entidade: LOC
palavra: Nº S/N
entidade: MISC
palavra: BARRAS
entidade: LOC
palavra: PI
entidade: LOC


In [ ]:
import pandas as pd

# fonte https://www.gov.br/trabalho-e-emprego/pt-br/assuntos/inspecao-do-trabalho/areas-de-atuacao/combate-ao-trabalho-escravo-e-analogo-ao-de-escravo
url = 'https://www.gov.br/trabalho-e-emprego/pt-br/assuntos/inspecao-do-trabalho/areas-de-atuacao/cadastro_de_empregadores.csv'

df = pd.read_csv(url, encoding='latin-1', sep=';', header=0, skiprows=5, skipfooter=0)

In [ ]:
# Remover colunas vazias
df = df.loc[:, df.notna().any()]
df.dropna(inplace=True)

<ipython-input-10-ae310941f469>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [ ]:
# dados usados para treino
df['Estabelecimento']

0      CERÂMICA J.A, RODOVIA PI, SENTIDO DE BARRAS A ...
1              FAZENZA BELA VISTA, ZONA RURAL, ITINGA/MA
2      SÍTIO TUMITAN, ZONA RURAL, ALFREDO MARCONDES/S...
3      RUA SARCANTO, 37, CASA 01, BAIRRO JARDIM FLOR ...
4      FAZENDA NATANAEL, ZONA RURAL, RIO PARDO DE MIN...
                             ...                        
466    FAZENDA RIO BRANCO, ESTRADA DA RUFINA, KM 11, ...
467    CÓRREGO DO MACUCO, DISTRITO DE PIRACEMA, AFONS...
468     LUNAPARK. GALPÃO NA RODOVIA BR-415 (AV. IBICA...
469    RUA FIRMINO MORGADO, Nº 293, BAIRRO JARDIM BRA...
472    FAZENDA TANQUES, ZONA RURAL, SÃO GONÇALO DO AB...
Name: Estabelecimento, Length: 468, dtype: object

### Treinando meu modelo customizado para encontrar apenas cidades e estado

In [ ]:
# meus dados de treino gerados no doccano (quantidade: 1)
TRAIN_DATA = [{'id': 87, 'text': 'CERÂMICA J.A, RODOVIA PI, SENTIDO DE BARRAS A CABACEIRAS, KM 7, Nº S/N, ZONA RURAL, BARRAS/PI', "label":[[84,90,"CITY"],[91,93,"STATE"]], 'Comments': []},]

In [3]:
# mais dados de treino gerados no doccano (quantidade: 29)
TRAIN_DATA = [{'id': 2823, 'text': 'SÍTIO TUMITAN, ZONA RURAL, ALFREDO MARCONDES/SP', 'label': [[27, 44, 'CITY'], [45, 47, 'STATE']], 'Comments': []}, {'id': 2824, 'text': 'RUA SARCANTO, 37, CASA 01, BAIRRO JARDIM FLOR DE MAIO, SÃO PAULO/SP', 'label': [[55, 64, 'CITY'], [65, 67, 'STATE']], 'Comments': []}, {'id': 2825, 'text': 'FAZENDA NATANAEL, ZONA RURAL, RIO PARDO DE MINAS/MG', 'label': [[30, 48, 'CITY'], [49, 51, 'STATE']], 'Comments': []}, {'id': 2826, 'text': 'SÍTIO BELA VISTA I, S/N, ZONA RURAL, ENTRE O DISTRITO DO ENTRONCAMENTO (CARNAUBAIS/RN) E O MUNICÍPIO DE ALTO RODRIGUES, CARNAUBAIS/RN.', 'label': [[120, 130, 'CITY'], [131, 133, 'STATE']], 'Comments': []}, {'id': 2827, 'text': 'GARIMPO DE AMETISTA, RODOVIA BA-210, SERRA DA QUIXABA, PARQUE NACIONAL DO BOQUEIRÃO DA ONÇA, ZONA RURAL, SENTO SÉ/BA', 'label': [[105, 113, 'CITY'], [114, 116, 'STATE']], 'Comments': []}, {'id': 2828, 'text': 'FAZENDA BAHIANA, CAMPO DE JACÓ, PROJETO ALASCA, ZONA RURAL, SANTA LUZIA/BA.', 'label': [[60, 71, 'CITY'], [72, 74, 'STATE']], 'Comments': []}, {'id': 2822, 'text': 'FAZENZA BELA VISTA, ZONA RURAL, ITINGA/MA', 'label': [[32, 38, 'CITY'], [39, 41, 'STATE']], 'Comments': []}, {'id': 2821, 'text': 'CERÂMICA J.A, RODOVIA PI, SENTIDO DE BARRAS A CABACEIRAS, KM 7, Nº S/N, ZONA RURAL, BARRAS/PI.', 'label': [[84, 90, 'CITY'], [91, 93, 'STATE']], 'Comments': []}, {'id': 2829, 'text': 'GARIMPO DO SALOMÃO - FAZENDA SERRA RICA - VILA CANGAIA, ZONA RURAL, CUMARU DO NORTE/PA', 'label': [[68, 83, 'CITY'], [84, 86, 'STATE']], 'Comments': []}, {'id': 2830, 'text': 'FAZENDA PEDRA BRANCA, ZONA RURAL, NOVA IPIXUNA/PA', 'label': [[34, 46, 'CITY'], [47, 49, 'STATE']], 'Comments': []}, {'id': 2831, 'text': 'FAZENDA PRIMAVERA, NOVA IPIXUNA/PA', 'label': [[19, 31, 'CITY'], [32, 34, 'STATE']], 'Comments': []}, {'id': 2832, 'text': 'ALOJAMENTO E PONTOS DE PROSTITUIÇÃO DE PROFISSIONAIS DO SEXO, NA CIDADE DE RIBEIRÃO PRETO/SP.', 'label': [[75, 89, 'CITY'], [90, 92, 'STATE']], 'Comments': []}, {'id': 2833, 'text': 'ROD GO 164, FAZENDA SÃO FRANCK, ZONA RURAL, ACREÚNA/GO.', 'label': [[44, 51, 'CITY'], [52, 54, 'STATE']], 'Comments': []}, {'id': 2834, 'text': 'ROD RS 324 KM 270.5, VILA ZUCCHETTI, NOVA ARAÇA/RS', 'label': [[37, 47, 'CITY'], [48, 50, 'STATE']], 'Comments': []}, {'id': 2835, 'text': 'AGROCOL, RODOVIA BR-050, PRÓXIMO AO KM 158, SENTIDO UBERABA-UBERLÂNDIA, UBERABA/MG', 'label': [[72, 79, 'CITY'], [80, 82, 'STATE']], 'Comments': []}, {'id': 2836, 'text': 'AVENIDA CONSELHEIRO FURTADO, 76, BATISTA CAMPOS, BELÉM/PA', 'label': [[49, 54, 'CITY'], [55, 57, 'STATE']], 'Comments': []}, {'id': 2838, 'text': 'FAZENDA FORMOSO II, ZONA RURAL, BURITIZEIRO/MG', 'label': [[32, 43, 'CITY'], [44, 46, 'STATE']], 'Comments': []}, {'id': 2839, 'text': 'CARNAUBAIS, LOCALIZADOS NO SÍTIO PANON 1, ZONA RURAL, CEP 59.650-000, AÇU/RN.', 'label': [[70, 73, 'CITY'], [74, 76, 'STATE']], 'Comments': []}, {'id': 2840, 'text': 'FAZENDA SÃO JUDAS TADEU, ZONA RURAL, UBERABA/MG', 'label': [[37, 44, 'CITY'], [45, 47, 'STATE']], 'Comments': []}, {'id': 2841, 'text': 'PROPRIEDADE RURAL, LOCALIZADA EM PICADA SÃO PAULO, ZONA RURAL, MORRO REUTER/RS.', 'label': [[62, 75, 'CITY'], [76, 78, 'STATE']], 'Comments': []}, {'id': 2842, 'text': 'RODOVIA BR 030, KM 92, SENTIDO CHAPADA GAÚCHA A MONTALVÂNIA, SN, ZONA RURAL, BONITO DE MINAS/MG', 'label': [[77, 92, 'CITY'], [93, 95, 'STATE']], 'Comments': []}, {'id': 2843, 'text': 'FÁBRICA , ALOJAMENTO E DEMAIS INSTALAÇÕES JUNTO A SEDE DA IGREJA LOCALIZADA NA QNM 5 CONJ. O, LOTE 31, CEILÂNDIA, BRASÍLIA/DF', 'label': [[114, 122, 'CITY'], [123, 125, 'STATE']], 'Comments': []}, {'id': 2844, 'text': 'PEDREIRA VEREDA, RODOVIA PI-248, S/N, ZONA RURAL, FLORES DO PIAUÍ/PI', 'label': [[50, 65, 'CITY'], [66, 68, 'STATE']], 'Comments': []}, {'id': 2845, 'text': 'FAZENDA BARRA DO SALITRE, ZONA RURAL, PATROCÍNIO/MG', 'label': [[38, 48, 'CITY'], [49, 51, 'STATE']], 'Comments': []}, {'id': 2846, 'text': 'FAZENDA BARRA DA CACHOEIRINHA, SÃO SEBASTIÃO, ZONA RURAL, BRASÍLIA/DF', 'label': [[58, 66, 'CITY'], [67, 69, 'STATE']], 'Comments': []}, {'id': 2847, 'text': 'FAZENDA ESPERANÇA, RODOVIA TO 080, ZONA RURAL, PARAÍSO/TO', 'label': [[47, 54, 'CITY'], [55, 57, 'STATE']], 'Comments': []}, {'id': 2848, 'text': 'CENTRO, PARANAVAÍ/PR', 'label': [[8, 17, 'CITY'], [18, 20, 'STATE']], 'Comments': []}, {'id': 2849, 'text': 'BAIRRO BELA VISTA, CAMPO BOM/RS.', 'label': [[19, 28, 'CITY'], [29, 31, 'STATE']], 'Comments': []}, {'id': 2850, 'text': 'ALOJAMENTO E PONTOS DE PROSTITUIÇÃO DE PROFISSIONAIS DO SEXO NA CIDADE DE RIBEIRÃO PRETO/SP', 'label': [[74, 88, 'CITY'], [89, 91, 'STATE']], 'Comments': []}]

In [4]:
import spacy
import random
from spacy.training.example import Example

# Carregue o modelo em branco do spaCy
nlp = spacy.blank("pt")

# Cria um componente NER (name entity recognition) vazio e adicione-o ao pipeline
ner = nlp.add_pipe("ner")

# Inicializa o 'treinador'
optimizer = nlp.begin_training()

In [10]:
# Loop de treinamento
for iteration in range(100):
  # Embaralha os dados de treinamento (TRAIN_DATA) a cada iteraçã
    random.shuffle(TRAIN_DATA)
    losses = {}
    for dic in TRAIN_DATA:
      # Cria um dicionário no formato necessário para o treinamento do spaCy, onde 'entities'
      #  contém as informações sobre as entidades presentes no texto.
        dicionario = {"entities": dic['label']}
        # Cria um exemplo de treinamento usando Example.from_dict
        example = Example.from_dict(nlp.make_doc(dic['text']), dicionario)
        # Atualiza o modelo com o exemplo de treinamento usando nlp.update
        # O parâmetro drop=0.5 indica que 50% dos gradientes serão descartados durante o treinamento, 
        # o que ajuda a evitar overfitting
        nlp.update([example], drop=0.5, losses=losses)
        # losses armazena as perdas (loss) durante o treinamento.
    print(losses)


{'ner': 1.9999288121333676}
{'ner': 0.0022332629232846113}
{'ner': 2.8131901467760336}
{'ner': 1.9093920237868283}
{'ner': 7.049812695339694}
{'ner': 0.1278002378001104}
{'ner': 0.33908033391150766}
{'ner': 4.0904129481729465e-09}
{'ner': 0.10370754849641647}
{'ner': 2.4696417259028663e-08}
{'ner': 0.0001323241669843299}
{'ner': 2.338408619565555e-08}
{'ner': 0.057605543015026274}
{'ner': 3.941203800492424}
{'ner': 0.24340946043957115}
{'ner': 3.7324215976878437}
{'ner': 1.9278717454087544}
{'ner': 0.5571812739530403}
{'ner': 1.2066301628491378}
{'ner': 1.9885779619235238}
{'ner': 0.02294171233971414}
{'ner': 0.0034086965216392233}
{'ner': 3.982321815964601e-06}
{'ner': 0.0023812360431789445}
{'ner': 0.07478294216895148}
{'ner': 0.19497040307985838}
{'ner': 1.541572756457579}
{'ner': 0.0010982870786989088}
{'ner': 0.7634688453686671}
{'ner': 2.007775291117065}
{'ner': 0.2263891473499463}
{'ner': 0.0003177261843135507}
{'ner': 1.7184749501197454e-05}
{'ner': 0.009459862271557285}
{'ner'

In [8]:
# Teste
texto_de_teste = "FAZENDA DAS PALMEIRAS, NA ZONA RURAL, RIO VERMELHO/MG"

# Processar texto de teste
doc = nlp(texto_de_teste)

# Extrair entidades nomeadas reconhecidas
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

RIO VERMELHO 38 50 CITY
MG 51 53 STATE


In [9]:
texto_de_teste = "rua paraíba, nº 223,  Volta Redonda/RJ"

doc = nlp(texto_de_teste)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Redonda 28 35 CITY
RJ 36 38 STATE
